# Mini project
### Tasks:

- Etap 1: Wczytaj dane, sprawdź ich strukturę i wykonaj wstępną eksplorację.
- Etap 2: filtruj dane sprzedażowe wg wartości i regionu
- Etap 3: policz metryki (np. średni zysk, liczba transakcji) dla wybranych kolumn
- Etap 4: oczyść dane klienta – brakujące dane, duplikaty, błędy
- Etap 5: scal kilka źródeł w jedną tabelę (np. zamówienia + klienci)

Wszystkie kroki, bedą realizowane na poniższym datasetcie.

In [63]:
# Dataset

import pandas as pd

# Dane sprzedażowe
df_sales = pd.DataFrame({
    'ID': [101, 102, 103, 104, 105, 106],
    'Sprzedawca': ['Anna', 'Bartek', 'Anna', 'Kasia', 'Marek', 'Bartek'],
    'Produkt': ['Laptop', 'Tablet', 'Smartfon', 'Laptop', 'Tablet', 'Laptop'],
    'Region': ['Północ', 'Południe', 'Północ', 'Zachód', 'Wschód', 'Południe'],
    'Sprzedaż': [3000, 1500, 2200, 2700, 1800, 3100],
    'Klient_ID': [1, 2, 1, 3, 4, 2]
})

# Dane klientów
df_clients = pd.DataFrame({
    'Klient_ID': [1, 2, 3, 4],
    'Nazwa': ['Firma A', 'Firma B', 'Firma C', 'Firma D'],
    'Miasto': ["Warszawa", 'Kraków', 'Gdańsk', 'Poznań'],
    'Branża': ['IT', 'Finanse', 'Edukacja', 'Budownictwo']
})

---

### Data exploration

In [64]:
df_sales

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
0,101,Anna,Laptop,Północ,3000,1
1,102,Bartek,Tablet,Południe,1500,2
2,103,Anna,Smartfon,Północ,2200,1
3,104,Kasia,Laptop,Zachód,2700,3
4,105,Marek,Tablet,Wschód,1800,4
5,106,Bartek,Laptop,Południe,3100,2


In [65]:
df_clients

,Klient_ID,Nazwa,Miasto,Branża
0,1,Firma A,Warszawa,IT
1,2,Firma B,Kraków,Finanse
2,3,Firma C,Gdańsk,Edukacja
3,4,Firma D,Poznań,Budownictwo


In [66]:
df_sales.info(verbose=True,
              memory_usage=False,
              show_counts=True)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          6 non-null      int64 
 1   Sprzedawca  6 non-null      object
 2   Produkt     6 non-null      object
 3   Region      6 non-null      object
 4   Sprzedaż    6 non-null      int64 
 5   Klient_ID   6 non-null      int64 
dtypes: int64(3), object(3)

In [67]:
df_clients.info(verbose=True,
                memory_usage=False,
                show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Klient_ID  4 non-null      int64 
 1   Nazwa      4 non-null      object
 2   Miasto     4 non-null      object
 3   Branża     4 non-null      object
dtypes: int64(1), object(3)

In [82]:
df_sprzedaz = df_sales['Sprzedaż'].describe(percentiles=[0.25, 0.5, 0.75])
df_sprzedaz

count       6.000000
mean     2383.333333
std       655.489639
min      1500.000000
25%      1900.000000
50%      2450.000000
75%      2925.000000
max      3100.000000
Name: Sprzedaż, dtype: float64

In [69]:
# Usseles for that dataframe
# df_clients.describe()

In [70]:
unique_products = df_sales["Produkt"].unique()
unique_region = df_sales['Region'].unique()
unique_industry = df_clients['Branża'].unique()
unique_seller = df_sales['Sprzedawca'].unique()

print(f"""There are:
    - {len(unique_products)} unique products: {list(unique_products)}
    - {len(unique_region)} unique regions: {list(unique_region)}
    - {len(unique_seller)} unique sellers: {list(unique_seller)}
    - {len(unique_industry)} unique industries: {list(unique_industry)}
""")

There are:
    - 3 unique products: ['Laptop', 'Tablet', 'Smartfon']
    - 4 unique regions: ['Północ', 'Południe', 'Zachód', 'Wschód']
    - 4 unique sellers: ['Anna', 'Bartek', 'Kasia', 'Marek']
    - 4 unique industries: ['IT', 'Finanse', 'Edukacja', 'Budownictwo']



In [71]:
# Check if there are any Null values
print(f"""In the dataframe \"df_sales\" {f"there {"is" if df_sales.isna().sum().sum() == 1 else "are"} {df_sales.isna().sum().sum()}" if df_sales.isna().values.any() else "there are not"} null value.
In the dataframe \"df_clients\" {f"there {"is" if df_clients.isna().sum().sum() == 1 else "are"} {df_clients.isna().sum().sum()}" if df_clients.isna().values.any() else "there are not"} null value."
      """)



In the dataframe "df_sales" there are not null value.
In the dataframe "df_clients" there are not null value."
      


In [193]:
# Best sellers, and their amout of sales
df_sales_grouped = df_sales.groupby(by=['Sprzedawca'], sort=True)
count_sells_by_seller = df_sales_grouped['Sprzedaż'].count()
amout_sells_by_seller = df_sales_grouped['Sprzedaż'].sum()


best_sellers_names = list((count_sells_by_seller[count_sells_by_seller >= count_sells_by_seller.quantile(0.95)]).keys())
best_sellers_amount = amout_sells_by_seller[best_sellers_names].to_dict()


In [229]:
# Sprzedaz na produkt
sum_sales_per_product = df_sales.groupby("Produkt")['Sprzedaż'].sum()
count_sales_per_product = df_sales.groupby("Produkt")['Sprzedaż'].count()
best_selling_product = count_sales_per_product.index[0]

best_selling_product 
best_selling_product_amout = sum_sales_per_product[best_selling_product]


In [242]:
df_merged = pd.merge(df_clients, df_sales)
df_merged

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
0,1,Firma A,Warszawa,IT,101,Anna,Laptop,Północ,3000
1,1,Firma A,Warszawa,IT,103,Anna,Smartfon,Północ,2200
2,2,Firma B,Kraków,Finanse,102,Bartek,Tablet,Południe,1500
3,2,Firma B,Kraków,Finanse,106,Bartek,Laptop,Południe,3100
4,3,Firma C,Gdańsk,Edukacja,104,Kasia,Laptop,Zachód,2700
5,4,Firma D,Poznań,Budownictwo,105,Marek,Tablet,Wschód,1800


In [245]:
df_merged.groupby(by='Region')['Sprzedaż'].sum()

Region
Południe    4600
Północ      5200
Wschód      1800
Zachód      2700
Name: Sprzedaż, dtype: int64

In [248]:
df_merged.loc[df_merged['Nazwa'] == df_merged.groupby(by='Nazwa')['Sprzedaż'].sum().index[0]]['Produkt']

0      Laptop
1    Smartfon
Name: Produkt, dtype: object

In [249]:
d1 = {name: str(dftype) for name, dftype in zip(df_sales.columns, df_sales.dtypes)}
d2 = {name: str(dftype) for name, dftype in zip(df_clients.columns, df_clients.dtypes)}
print(f"""Conclusion:
      We've got two datasets: ['df_sales', 'df_clients']
      
      The first dataset consists {len(df_sales.columns)} columns: {d1},
      and in the \"df_sales\" {f"there {"is" if df_sales.isna().sum().sum() == 1 else "are"} {df_sales.isna().sum().sum()}" if df_sales.isna().values.any() else "there are 0"} null value.
      About data:
      Mean sale is equal to {df_sprzedaz.__getitem__('mean')}.
      Median sale was equal to {df_sprzedaz.__getitem__('50%')}
      Minimum sale was equal to {df_sprzedaz.__getitem__('min')} and the maximum was {df_sprzedaz.__getitem__('max')}.
      We've got {len(df_sales['Sprzedawca'].unique())} sellers: {list(df_sales['Sprzedawca'].unique())} who took part in {len(df_sales['Sprzedawca'])} sales.
      We can observe {len(df_sales['Produkt'].unique())} unique product, which are: {list(df_sales['Produkt'].unique())}
      Also Sales are distributed between {len(df_sales['Region'].unique())} regions, like {list(df_sales['Region'].unique())}

      The second dataset consists {len(df_clients.columns)} columns: {d2}
      and in the \"df_clients\" {f"there {"is" if df_clients.isna().sum().sum() == 1 else "are"} {df_clients.isna().sum().sum()}" if df_clients.isna().values.any() else "there are 0"} null value.
      About data:
      We've got {len(df_clients['Nazwa'].unique())} clients from {len(df_clients['Branża'].unique())} industries: {list(df_clients['Branża'].unique())}, which operates on different cities: {list(df_clients['Miasto'])}.
      
      Additional info:
      We can highlight {len(best_sellers_names)} best sellers, which are {best_sellers_names}. They've sold sum of {best_sellers_amount}.
      Also worth mentioning is product review, the best selling product is {best_selling_product} with sum of sales equal to {best_selling_product_amout}.
      About the firms, we can highlight the {df_merged.groupby(by='Nazwa')['Sprzedaż'].sum().index[0]} which bought {df_merged.loc[df_merged['Nazwa'] == df_merged.groupby(by='Nazwa')['Sprzedaż'].sum().index[0]]['Produkt'].values} for the value of {df_merged.groupby(by='Nazwa')['Sprzedaż'].sum().iloc[0]}.
      """)

Conclusion:
      We've got two datasets: ['df_sales', 'df_clients']
      
      The first dataset consists 6 columns: {'ID': 'int64', 'Sprzedawca': 'object', 'Produkt': 'object', 'Region': 'object', 'Sprzedaż': 'int64', 'Klient_ID': 'int64'},
      and in the "df_sales" there are 0 null value.
      About data:
      Mean sale is equal to 2383.3333333333335.
      Median sale was equal to 2450.0
      Minimum sale was equal to 1500.0 and the maximum was 3100.0.
      We've got 4 sellers: ['Anna', 'Bartek', 'Kasia', 'Marek'] who took part in 6 sales.
      We can observe 3 unique product, which are: ['Laptop', 'Tablet', 'Smartfon']
      Also Sales are distributed between 4 regions, like ['Północ', 'Południe', 'Zachód', 'Wschód']

      The second dataset consists 4 columns: {'Klient_ID': 'int64', 'Nazwa': 'object', 'Miasto': 'object', 'Branża': 'object'}
      and in the "df_clients" there are 0 null value.
      About data:
      We've got 4 clients from 4 industries: ['IT', 'Finans

---

### Data filtering


In [ ]:
# Sales over 2500
df_sales.loc[df_sales['Sprzedaż'] >= 2500] # Sales over 2500

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
0,101,Anna,Laptop,Północ,3000,1
3,104,Kasia,Laptop,Zachód,2700,3
5,106,Bartek,Laptop,Południe,3100,2


In [ ]:
df_sales.loc[df_sales['Region'] == 'Północ'] # Sales in Region North

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
0,101,Anna,Laptop,Północ,3000,1
2,103,Anna,Smartfon,Północ,2200,1


In [ ]:
df_sales.loc[(df_sales['Region'].isin(['Północ', 'Zachód'])) & (df_sales['Produkt'] == 'Laptop')] # Sales in two regions where laptops were sold

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
0,101,Anna,Laptop,Północ,3000,1
3,104,Kasia,Laptop,Zachód,2700,3


In [267]:
df_sales.loc[df_sales['Sprzedaż'].between(2000, 3000)] # Sales ranging from 2000 to 3000

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
0,101,Anna,Laptop,Północ,3000,1
2,103,Anna,Smartfon,Północ,2200,1
3,104,Kasia,Laptop,Zachód,2700,3


In [271]:
df_sales.loc[(df_sales['Region'] == 'Południe') & (df_sales['Produkt'] == "Smartfon")]['Sprzedawca']

Series([], Name: Sprzedawca, dtype: object)

In [274]:
df_sales.loc[(~ (df_sales['Produkt'] == 'Laptop')) & (df_sales['Sprzedaż'] < 2000)]

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
1,102,Bartek,Tablet,Południe,1500,2
4,105,Marek,Tablet,Wschód,1800,4


In [275]:
df_merged

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
0,1,Firma A,Warszawa,IT,101,Anna,Laptop,Północ,3000
1,1,Firma A,Warszawa,IT,103,Anna,Smartfon,Północ,2200
2,2,Firma B,Kraków,Finanse,102,Bartek,Tablet,Południe,1500
3,2,Firma B,Kraków,Finanse,106,Bartek,Laptop,Południe,3100
4,3,Firma C,Gdańsk,Edukacja,104,Kasia,Laptop,Zachód,2700
5,4,Firma D,Poznań,Budownictwo,105,Marek,Tablet,Wschód,1800


In [ ]:
df_merged.loc[(df_merged['Miasto'] == 'Poznań') & (~ (df_merged['Produkt'] == 'Smarfon'))] 

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
5,4,Firma D,Poznań,Budownictwo,105,Marek,Tablet,Wschód,1800


In [279]:
df_merged.loc[(df_merged['Miasto'] == 'Warszawa') & (df_merged['Produkt'] == 'Laptop') & (df_merged['Sprzedaż'] > df_merged['Sprzedaż'].mean())]

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
0,1,Firma A,Warszawa,IT,101,Anna,Laptop,Północ,3000


In [281]:
df_merged.loc[(df_merged['Branża'] == 'IT') & (df_merged['Sprzedaż'] > 2000)]

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
0,1,Firma A,Warszawa,IT,101,Anna,Laptop,Północ,3000
1,1,Firma A,Warszawa,IT,103,Anna,Smartfon,Północ,2200


In [286]:
df_merged.loc[(df_merged['Produkt'] == 'Tablet') & (~(df_merged['Branża'] == 'Edukacja')), 'Sprzedawca']

2    Bartek
5     Marek
Name: Sprzedawca, dtype: object

---